In [1]:
%%capture
!uv pip install polars

In [2]:
import polars as pl

In [3]:
from kn1ght.constants import SPECIAL_TOKENS

In [4]:
CURRENT_DATASET = "/chess_games_2025-01-17.csv"
DATA_DIRECTORY = "../.data/"
CURRENT_DATASET_PATH = DATA_DIRECTORY + CURRENT_DATASET

In [5]:
df = pl.read_csv(CURRENT_DATASET_PATH, null_values=["None"])

df.head()

Date,Result,WhiteElo,BlackElo,PGN
str,str,i64,i64,str
"""2000.03.14""","""1-0""",2851,null,"""1.d4 d5 2.c4 e6 3.Nc3 Nf6 4.cx…"
"""2000.03.14""","""1-0""",2851,null,"""1.e4 d5 2.exd5 Qxd5 3.Nc3 Qa5 …"
"""1999.11.20""","""1-0""",2851,null,"""1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.…"
"""1999.11.20""","""1-0""",2851,null,"""1.e4 d5 2.exd5 Qxd5 3.Nc3 Qa5 …"
"""2000.02.20""","""1/2-1/2""",2851,2633,"""1.e4 e5 2.Nf3 Nc6 3.Bb5 a6 4.B…"


In [6]:
def test_train_validate_splits(data, directory) -> None:
    data[:] = (SPECIAL_TOKENS["START"] + d + SPECIAL_TOKENS["END"] for d in data)

    training_data_end_index = int(len(data) * 0.8)
    test_data_end_index = int(len(data) * 0.9)

    training_data = list(data[0:training_data_end_index])
    test_data = list(data[training_data_end_index:test_data_end_index])
    validation_data = list(data[test_data_end_index:])

    training_files = [training_data, test_data, validation_data]

    file_names = ["training_data", "test_data", "validation_data"]

    for i, f in enumerate(training_files):
        with open(f"{directory}/{file_names[i]}.txt", "w") as file:
            file.write("\n".join(f))
            print(f"{file_names[i]} created...")

In [7]:
all_training_data = pl.Series(df.select(pl.col("PGN").shuffle())).to_list()

test_train_validate_splits(all_training_data, DATA_DIRECTORY)

training_data created...
test_data created...
validation_data created...
